In [1]:
#load model
from ultralytics import YOLO

model = YOLO(r"D:\code\demo_day\license-plate-recognition\models\plate_detection.pt", verbose=False)

In [2]:
import os
from tqdm import tqdm
import cv2

# Hàm chuyển đổi từ tọa độ tỉ lệ (x_center, y_center, width, height) sang tọa độ (x1, y1, x2, y2) theo pixel
def convert_to_pixel_coordinates(label, img_width, img_height):
    x_center, y_center, w, h = label[1:]  # Bỏ nhãn đầu tiên
    x_center, y_center, w, h = x_center * img_width, y_center * img_height, w * img_width, h * img_height
    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)
    return x1, y1, x2, y2

# Hàm tính IoU giữa hai hộp giới hạn (bounding box)
def calculate_iou(boxA, boxB):
    # Tính tọa độ của vùng giao nhau (intersection)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Tính diện tích của vùng giao nhau
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Tính diện tích của hai hộp giới hạn
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # Tính diện tích của vùng hợp nhất (union)
    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) != 0 else 0
    
    return iou

iou_threshold = 0.3  # IoU threshold (you can adjust this)
conf_threshold = 0.5  # Confidence threshold (you can adjust this)

folder_path = r"D:\code\demo_day\license-plate-recognition\datasets\License Plate Recognition.v4-resized640_aug3x-accurate.yolov8\test"
image_paths = os.path.join(folder_path, "images")
labels_paths = os.path.join(folder_path, "labels")

total_iou = 0
iou_count = 0

for filename in tqdm(os.listdir(image_paths)):
    image_path = os.path.join(image_paths, filename)
    label_path = os.path.join(labels_paths, filename[:-3] + "txt")

    # Đọc kích thước ảnh
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    labels = []

    # Đọc và chuyển đổi nhãn từ tọa độ tỉ lệ sang pixel
    with open(label_path, 'r') as file:
        for line in file:
            values = list(map(float, line.split()))
            pixel_box = convert_to_pixel_coordinates(values, img_width, img_height)
            labels.append(pixel_box)
    
    # Dự đoán kết quả từ mô hình
    results = model(image_path, verbose=False, conf=conf_threshold, iou=iou_threshold)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Lấy tọa độ hộp từ mô hình theo pixel
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detected_box = (x1, y1, x2, y2)

            # Tính IoU giữa hộp dự đoán và nhãn
            for label_box in labels:
                iou = calculate_iou(detected_box, label_box)
                if iou > 0.5:
                    total_iou += iou
                    iou_count += 1

# Tính toán IoU trung bình
if iou_count > 0:
    average_iou = total_iou / iou_count
else:
    average_iou = 0

print(f"IoU trung bình trên toàn bộ dữ liệu: {average_iou}")


  0%|          | 0/1019 [00:00<?, ?it/s]

100%|██████████| 1019/1019 [00:26<00:00, 38.61it/s]

IoU trung bình trên toàn bộ dữ liệu: 0.8476255686700721


In [3]:
import os
from tqdm import tqdm
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed

folder_path = r"D:\code\demo_day\license-plate-recognition\datasets\License Plate Recognition.v4-resized640_aug3x-accurate.yolov8\all"

# Hàm chuyển đổi từ tọa độ tỉ lệ (x_center, y_center, width, height) sang tọa độ (x1, y1, x2, y2) theo pixel
def convert_to_pixel_coordinates(label, img_width, img_height):
    x_center, y_center, w, h = label[1:]  # Bỏ nhãn đầu tiên
    x_center, y_center, w, h = x_center * img_width, y_center * img_height, w * img_width, h * img_height
    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)
    return x1, y1, x2, y2

# Hàm tính IoU giữa hai hộp giới hạn (bounding box)
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) != 0 else 0
    return iou

# Hàm xử lý mỗi ảnh và tính toán IoU
def process_image(filename):
    image_path = os.path.join(image_paths, filename)
    label_path = os.path.join(labels_paths, filename[:-3] + "txt")

    # Đọc kích thước ảnh
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            values = list(map(float, line.split()))
            pixel_box = convert_to_pixel_coordinates(values, img_width, img_height)
            labels.append(pixel_box)

    # Dự đoán kết quả từ mô hình
    results = model(image_path, verbose=False, conf=conf_threshold, iou=iou_threshold)

    local_total_iou = 0
    local_iou_count = 0

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detected_box = (x1, y1, x2, y2)

            for label_box in labels:
                iou = calculate_iou(detected_box, label_box)
                if iou > 0.5:
                    local_total_iou += iou
                    local_iou_count += 1

    return local_total_iou, local_iou_count

# Thiết lập các đường dẫn và ngưỡng
iou_threshold = 0.3
conf_threshold = 0.5
image_paths = os.path.join(folder_path, "images")
labels_paths = os.path.join(folder_path, "labels")

# Khởi tạo tổng iou và bộ đếm
total_iou = 0
iou_count = 0

# Sử dụng ThreadPoolExecutor để xử lý đa luồng
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_image, filename) for filename in os.listdir(image_paths)]

    for future in tqdm(as_completed(futures), total=len(futures)):
        local_total_iou, local_iou_count = future.result()
        total_iou += local_total_iou
        iou_count += local_iou_count

# Tính toán IoU trung bình
if iou_count > 0:
    average_iou = total_iou / iou_count
else:
    average_iou = 0

print(f"IoU trung bình trên toàn bộ dữ liệu: {average_iou}")

100%|██████████| 24238/24238 [09:12<00:00, 43.88it/s]

IoU trung bình trên toàn bộ dữ liệu: 0.8724153379115345
